In [5]:
import pandas as pd
import numpy as np
from collections import defaultdict
import os, pyodbc, psycopg2,cx_Oracle
from mysql import *
from sqlalchemy import create_engine
from dateutil.parser import *
from datetime import *
from dateutil.relativedelta import *
import create_table.tbl_mysql as myq
import create_table.tbl_mssql as msq
from o_fnsql import *
from o_fnsql_2 import *


class OMSQ:
    def __init__(self, srv, user='',password='',host='',db=''):
        self.U = user
        self.P = password
        self.H = host
        self.D = db
        self.engin = ''
        self.cn = ''
        self.conn = ''
        self.dbtype = srv
        if srv.lower() == 'oracle':
            self.cn = cx_Oracle.connect('SOC_READ','soc_read', 'ossam-cluster-scan.robi.com.bd:1721/RBPB.robi.com.bd')
            print(self.cn.version)
        else:
            try:
                self.engin = create_engine(cstr(srv,user,password,host,db))
                self.conn = self.engin.connect()
                self.cn = self.engin.raw_connection()
                print(srv + ' conn succ')
            except:
                print(srv + ' conn failed')
        self.T = ''
        self.tbl = ''
        self.C = ''
        self.track = defaultdict(dict)
        self.insls = []
        self.updls = []
        self.updset = ''
        self.select = ''
        self.cond = ''
        self.qrydfmain = pd.DataFrame([])
        self.qrydf = pd.DataFrame([])
    def Addcond(self, addby= ' AND ',condition =''):
        if self.cond == '':
            self.cond = ' WHERE ' + condition
        else:
            self.cond = self.cond + addby + condition
        self.track = trc_dc(self.track,'S_addcond',[condition])
    def create_tbl(self,df=False,tbl=False):
        if df == True:
            if self.dbtype == 'mssql':
                msq.CreateTable_MSSQL(df, tbl, self.cn)
            elif self.dbtype == 'mysql':
                myq.CreateTable_MYSQL(connection = self.cn, tablename = tbl, df = df)
        else:
            if self.dbtype == 'mssql':
                msq.CreateTable_MSSQL(self.qrydfmain, self.tbl, self.cn)
            elif self.dbtype == 'mysql':
                myq.CreateTable_MYSQL(connection = self.cn, tablename = self.tbl, df = self.qrydfmain)
    def table_info(self, tbl=False):
        if tbl == False:
            tbl = self.tbl
        if self.dbtype == 'MSSQL':
            self.tableinfo = mssql_table_colinfo(tbl,self.conn)
        elif self.dbtype == 'MYSQL':
            self.tableinfo = mysql_table_colinfo(self.D, tbl, self.conn)
        else:
            print('No DB initiated yet')
            
class OMQRY(OMSQ):
    def S_check_cols(self, tbl):
        s1 = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE "
        s2 = "TABLE_NAME = '" + tbl + "' ORDER BY ORDINAL_POSITION"
        df = pd.read_sql(s1 + s2, con = self.engin)
        return df
    def S_select(self, tbl, col):
        self.T = tbl
        if type(col) is list:
            self.select = "select " + ','.join(map(str, col)) + " from " + self.T
        else:
            self.select = "select " + str(col) + " from " + self.T
        if self.cond !='':
            self.select = self.select + self.cond
        self.sells.append(self.select)
        self.track = trc_dc(self.track,'S_select',self.select)
        return self.select
    def S_cond_time(self, T1, T2, db_datetime_col='LASTOCCURRENCE', addby= ' AND '):
        if dbtype == 'oracle':
            if self.cond == '':
                self.cond = ' WHERE ' + timebetween(T1,T2,db_datetime_col)
            else:
                self.cond = self.cond + addby + timebetween(T1,T2,db_datetime_col)
        elif dbtype == 'mssql':
            if self.cond == '':
                self.cond = ' WHERE ' + timebetween(T1,T2,db_datetime_col,dbtype)
            else:
                self.cond = self.cond + addby + timebetween(T1,T2,db_datetime_col,dbtype)
        self.track = trc_dc(self.track,'S_addcond_time',[timebetween(T1,T2,db_datetime_col,dbtype)])


class S_EXPORT(OMSQ):
    def Tblcol(self,tbl,col):
        self.T = tbl
        self.C = col
    def S_insert(self,vals=[]):
        insert_into = FS_insert(self.T,self.C,vals=[])
        self.insls.append(insert_into)
        self.track = trc_dc(track,'S_insert',insert_into)
        return insert_into
    def S_fault_test(self,vals,condcol,operator=' = '):
        T0 = ' where ' + condcol + operator
        T1 = 'select * from ' + self.T + ' where ' + condcol + operator
        T2 = 'insert into ' + self.T + condcol + ' values '
        cursors = self.cn.cursor()
        for i,v in enumerate(vals):
            if self.C[i] == condcol:
                T0 = T0 + "'" + str(v) + "'"
                T1 = T1 + "'" + str(v) + "'"
                T2 = T2 + "'" + str(v) + "'"
                print('T0-T2 -- ',T0,T1,T2)
                try:
                    cursors.execute(T1)
                    print(T1)
                except:
                    cursors.execute(T2)
                    print(T2)
                finally:
                    break
        else:
            self.conn.commit()
            cursors.close()
        T31 = 'UPDATE ' + self.T + ' SET ' 
        for i,v in enumerate(vals):
            if self.C[i] != condcol:
                T32 = T31 + self.C[i] + "='" + str(v) + "'" + T0
                T42 = T31 + self.C[i] + "='" + str(TM1(v)) + "'" + T0
                try:
                    cursors.execute(T32)
                    print('pass: ', T32)
                except:
                    try:
                        cursors.execute(T42)
                        print('pass: ', T42)
                    except:
                        print('Failed: ', T32, " ~~ ", T42)
    def S_update(self,vals,condcol=[],operator=' LIKE ',condop =' and ',enable_condcol=True):
        self.updset = FS_update(vals,condcol,operator,condop,enable_condcol=True)
        self.updls.append(self.upd_set)
        self.track = trc_dc(self.track,'S_update',self.upd_set)
        return self.updset

class B_EXPORT(OMSQ):
    def export_prep(self, ndf, tbl, bycol=[], sep=','):
        self.tbl = tbl
        self.table_info(self.tbl)
        df = ndf.fillna('$')
        df = mod_datetime(df)
        self.qrydfmain = df
        df = rmv_space_in_cols(self.qrydfmain)
        ls1 = insert_into(df)
        lss1 = ['INSERT INTO ' + tbl + i for i in ls1]
        if bycol is None or bycol==[]:
            df1 = pd.DataFrame(ls1, columns=['INS'])
            df1['INS'] = df1.apply(lambda x : 'INSERT INTO ' + tbl + x['INS'], axis = 1)
            self.qrydf = df1
        else:
            ls2 = update_into(df,bycols=bycol)
            dfn = df[bycol]
            ls3 = update_into(dfn,bycols=[],seperator=sep,operator='=')
            lss2 = ['UPDATE ' + tbl + ' SET ' + i + ' WHERE ' + j for i, j in zip(ls2, ls3)]
            self.qrydf = pd.DataFrame(zip(lss2,lss1),columns=['UPD','INS'])
    def export_execute(self, ifexist='fail'):
        lsfail = []
        try:
            self.qrydfmain.to_sql(self.tbl, con=self.engin, if_exists=ifexist, chunksize=10000)
            print('try success')
        except:
            self.create_tbl()
            for i in range(len(self.qrydf)):
                for j in self.qrydf:
                    q = self.qrydf.loc[i,j]
                    x = q.find(" WHERE ")
                    if x != -1:
                        qq = 'select 1 from ' + self.tbl + ' ' + q[x:len(q)]
                        ddf = pd.read_sql(qq,con=self.engin)
                        if ddf.shape[0]>0:
                            try:
                                self.conn.execute(q)
                                break
                            except:
                                lsfail.append(q)
                    else:
                        try:
                            self.conn.execute(q)
                            break
                        except:
                            lsfail.append(q)
            else:
                print(lsfail)


                
def csv2df(csvfl): 
    ndf = pd.read_csv(csv1)
    xdf = mod_datetime(ndf)
    df = rmv_space_in_cols(xdf)
    return df

csv1 = "D:\\omEngin\\AOmPy_o_\\csv_o_\\sclick3.csv"
ndf = csv2df(csv1)
col = ndf.columns.to_list()
val = ndf.values.tolist()

#x = B_EXPORT('mssql',user='sa',password='1q2w3eaz$',host='192.168.0.102',db='omdb')
#x = B_EXPORT('mysql',user='root',password='admin',host='127.0.0.1:3306',db='odb')
#x.export_prep(ndf,'tbl40',['Serial'])
#x.export_execute()
#
x= OMQRY('mssql',user='sa',password='1q2w3eaz$',host='192.168.0.102',db='omdb')
x.S_check_cols('tbl40')







mssql conn failed


OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')
(Background on this error at: http://sqlalche.me/e/13/e3q8)